In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.pylab import rcParams
get_ipython().run_line_magic('matplotlib', 'inline')
sns.set(style='white', context='notebook', palette='deep')
import warnings
warnings.filterwarnings('ignore')


In [63]:
# Read the datasets & knowing the shape of each dataset
alldata = pd.read_csv("report.csv")

submission = pd.read_csv("submission.csv")

In [65]:
##duplicates investigation
#酪農場代號(A:桃園楊梅，B:彰化福興，C:屏東萬丹)
report_label={1:"ID", 2:"資料年度", 3:"資料月份", 4:"酪農場代號", 5:"乳牛編號", 6:"父親牛精液編號",
              7:"母親乳牛編號", 8:"出生日期", 9:"胎次", 10:"泌乳天數", 11:"乳量", 12:"最近分娩日期",
              13:"採樣日期", 14:"月齡", 15:"檢測日期", 16:"最後配種日期", 17:"最後配種精液", 18:"配種次數",
              19:"前次分娩日期", 20:"第一次配種日期", 21:"第一次配種精液"}


# alldata.head()
# report_dup=report[report["5"].duplicated() ==2019 #5 = 乳牛編號
#report_dup.sort_values(by=["12"])
# reason for duplication: multiple samples per birth

In [66]:
#Sale price is our goal for prediction, investated as prioity
# alldata['11'].describe()

In [67]:
#graph
# f, ax = plt.subplots(figsize=(10, 7.5))
# sns.distplot(alldata['11'], rug=True)

In [68]:
data_id = alldata[alldata['11'].isna()] #test_data is all rows with "乳量" column IS NaN
print(data_id.shape)
data_id = data_id["1"] #drop "乳量" column
print(data_id.shape)

(4263, 21)
(4263,)


In [39]:
##drop columns that are date or identification
alldata = alldata.drop(alldata.columns[[0,1,2,4,5,6,7,11,12,14,15,16,18,19,20]],axis=1)
# '5', '6', '7', '17', 21 try to leave out
alldata = alldata.reset_index(drop=True)
print(alldata.shape)

(37517, 6)


In [40]:
#count all missing values of each columns
missing=[]
for col in alldata.columns:
    if alldata[col].isnull().sum() > 0:
        missing.append(col)
        print(col, " : ", alldata[col].isnull().sum())

10  :  1
11  :  4263


In [41]:
##function to fill NaN

def fill_nan(data):
    #data['6'] = data['6'].fillna(0)
    #data['7'] = data['7'].fillna(0)
    data['10'] = data['10'].fillna(data['10'].mode()[0]) 
    #data['17'] = data['17'].fillna(0)
    #data['21'] = data['21'].fillna(0)
    return data

In [42]:
alldata = fill_nan(alldata)

In [49]:
#one hot encoding for columns with categorical value 
alldata = pd.get_dummies(alldata)

##get test data set according to NAN (4263 rows = submission row size)
test_data = alldata[alldata['11'].isna()] #test_data is all rows with "乳量" column IS NaN
test_data = test_data.drop(['11'],axis=1) #drop "乳量" column
train_data = alldata.dropna(subset=['11']) #drop cell with NaN "乳量"
print(alldata.shape)
print(test_data.shape)
print(train_data.shape)

(37517, 8)
(4263, 7)
(33254, 8)


In [45]:
# feature scaling - normalization
from sklearn import preprocessing
train_data = preprocessing.normalize(train_data[['9', '10',  '14', '18']])

In [73]:
##normalized "乳量" seprately because of the NaN cells --- but don't do this, it normalizes y
#preprocessing.normalize(train_data[[ '11']])

In [50]:
train_data.shape
#alldata.head()

(33254, 8)

In [51]:
## x_data from report file without NaN; y_data = "乳量"

x_data = train_data.drop(['11'],axis=1) #drop "乳量" column for x_data
y_data = train_data["11"] #y_data = "乳量"

#Make into machine readable numpy array
x_data = np.array(x_data)
y_data = np.array(y_data)
x_data.shape

(33254, 7)

In [52]:
##split train & validaton set

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data,
                                                    test_size=0.1,
                                                    random_state=10)

In [54]:
print(y_train.shape)
print(x_train.shape)

(29928,)
(29928, 7)


In [55]:
#training data witg XGBoost model
#build model
import xgboost as xgb

dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_test)

params = {"colsample_bytree":0.4603, "gamma":0.0468, 
          "learning_rate":0.05, "max_depth":3, 
          "min_child_weight":1.7817, "n_estimators":2200,
          "reg_alpha":0.4640, "reg_lambda":0.8571,
          "subsample":0.5213, "silent":1,
          "nthread":-1}

model = xgb.cv(params, dtrain, num_boost_round=500, early_stopping_rounds=100)

In [56]:
#train with model
from sklearn import metrics

xgb_f = xgb.XGBRegressor(**params) #the params were tuned using xgb.cv

xgb_f.fit(x_train, y_train)
pred= xgb_f.predict(x_test)
print(pred)


[23.972408 27.461594 17.674353 ... 21.76603  19.611643 25.516203]


In [57]:
print(y_train.size)
print(pred.size)

##see RMSE in validation set
print("RMSE: %.2f" % np.sqrt(np.mean((pred-y_test)**2)))
print(pred.shape)

29928
3326
RMSE: 6.39


In [69]:
#use model for prediction
test_data= np.array(test_data)
print(test_data.shape)
pred_test= xgb_f.predict(test_data)
print(pred_test.shape)
print(data_id.shape)

(4263, 7)
(4263,)
(4263,)


In [72]:
#sub = pd.read_csv('./sample_submission.csv')
test_submission = pd.DataFrame({'ID':data_id, '1': pred_test})
test_submission.to_csv('./test_submission.csv', index = False)